In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cmocean
import netCDF4
from datetime import datetime, timedelta

# Mapping tools. Hard to install. Be sure to install *all packages* with conda 
# using the channel 'conda-forge'
import cartopy.crs as ccrs
import cartopy.feature

# xarray is a package for working with geophysical datasets, including netcdf
# See: http://xarray.pydata.org/en/stable/  for documentation
import xarray

/Users/rob/miniconda3/envs/core/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
# data location can be the threeds server url, or a local netcdf file
loc = 'http://barataria.tamu.edu:8080/thredds/dodsC/NcML/txla_hindcast_agg'

In [3]:
xr = xarray.open_dataset(loc)
xr

<xarray.Dataset>
Dimensions:         (eta_psi: 190, eta_rho: 191, eta_u: 191, eta_v: 190, ocean_time: 222767, s_rho: 30, s_w: 31, tracer: 6, xi_psi: 670, xi_rho: 671, xi_u: 670, xi_v: 671)
Coordinates:
  * s_rho           (s_rho) float64 -0.9833 -0.95 -0.9167 -0.8833 -0.85 ...
  * s_w             (s_w) float64 -1.0 -0.9667 -0.9333 -0.9 -0.8667 -0.8333 ...
    lon_rho         (eta_rho, xi_rho) float64 ...
    lat_rho         (eta_rho, xi_rho) float64 ...
    lon_u           (eta_u, xi_u) float64 ...
    lat_u           (eta_u, xi_u) float64 ...
    lon_v           (eta_v, xi_v) float64 ...
    lat_v           (eta_v, xi_v) float64 ...
    lon_psi         (eta_psi, xi_psi) float64 ...
    lat_psi         (eta_psi, xi_psi) float64 ...
  * ocean_time      (ocean_time) datetime64[ns] 1993-01-01T01:00:00 ...
Dimensions without coordinates: eta_psi, eta_rho, eta_u, eta_v, tracer, xi_psi, xi_rho, xi_u, xi_v
Data variables:
    ntimes          int32 ...
    ndtfast         int32 ...
    dt     

In [9]:
# select a time using the 'sel' method

salt = xr.salt.sel(ocean_time='2008-07-15 00:00') # a single time slice
# salt = xr.salt.sel(ocean_time='2008-07-15') # an entire day
# salt = xr.salt.sel(ocean_time='2008-07') # an entire month
salt

<xarray.DataArray 'salt' (ocean_time: 744, s_rho: 30, eta_rho: 191, xi_rho: 671)>
[2860553520 values with dtype=float32]
Coordinates:
  * s_rho       (s_rho) float64 -0.9833 -0.95 -0.9167 -0.8833 -0.85 -0.8167 ...
    lon_rho     (eta_rho, xi_rho) float64 ...
    lat_rho     (eta_rho, xi_rho) float64 ...
  * ocean_time  (ocean_time) datetime64[ns] 2008-07-01 2008-07-01T01:00:00 ...
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:    salinity
    time:         ocean_time
    field:        salinity, scalar, series
    _ChunkSizes:  [  1  15  96 336]

In [5]:
# or, select a range of times using slice

salt = xr.salt.sel(ocean_time=slice('2008-07-10','2008-07-20'))
salt.ocean_time

<xarray.DataArray 'ocean_time' (ocean_time: 264)>
array(['2008-07-10T00:00:00.000000000', '2008-07-10T01:00:00.000000000',
       '2008-07-10T02:00:00.000000000', ..., '2008-07-20T21:00:00.000000000',
       '2008-07-20T22:00:00.000000000', '2008-07-20T23:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * ocean_time  (ocean_time) datetime64[ns] 2008-07-10 2008-07-10T01:00:00 ...
Attributes:
    long_name:    time since initialization
    field:        time, scalar, series
    _ChunkSizes:  524288

In [7]:
# write a slice to a local netCDF file
# ...this takes a bit.

xr.salt.sel(ocean_time=slice('2008-07-10','2008-07-11')).to_netcdf('test.nc')

In [6]:
# Evaluation is lazy, and no data is pulled until you ask for it. 
# Get it by explicitly getting the 'values', that converts to a numpy array
# Note, pulling lots of data from the thredds server can be slow.

salt = xr.salt.sel(ocean_time='2008-07-15 00:00')
salt.values

array([[[ 35.16997147,  35.17002487,  35.16907883, ...,  34.91418076,
          34.91830826,  34.94934082],
        [ 35.16991425,  35.16873932,  35.16786194, ...,  34.96037674,
          34.97167969,  34.98048782],
        [ 35.16848373,  35.16723633,  35.16622162, ...,  35.02756882,
          35.04846191,  35.06291199],
        ..., 
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan]],

       [[ 35.11896896,  35.11919785,  35.11816406, ...,  34.99176407,
          35.00452423,  35.0553093 ],
        [ 35.11873627,  35.11783981,  35.11698151, ...,  35.06080627,
          35.08478546,  35.10634995],
        [ 35.11702347,  35.11607742,  35.11529922, ...,  35.16189957,
          35.19302368,  35.21157455],
        ...,

In [ ]:
# load in some data used for plotting

# get depths, lon, and lat points
h = xr.h.values
lon = xr.lon_rho.values
lat = xr.lat_rho.values

# sea surface salinity
sss = xr.salt.sel(ocean_time='2008-07-15 00:00').isel(s_rho=-1).values

In [ ]:
# Here is an example of how to plot on a map

# Set the projection
projection = ccrs.LambertConformal(central_longitude=-92.0, central_latitude=28.0,
                                   standard_parallels=(26.0, 30.0), cutoff=20)

# initialize the figure and axes objects
fig = plt.figure(figsize=(12, 5))
ax = fig.add_axes([0, 0, 1, 1], projection=projection)

# Natural earth is the best.
land_10m = cartopy.feature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='face', facecolor='0.5')
ax.coastlines('10m')
ax.add_feature(land_10m)

# Add bathymetric contours at standard depths
ax.contour(lon, lat, h, [10, 20, 50, 100, 200, 500, 1000], colors='k', 
           linewidths=0.2, transform=ccrs.PlateCarree())

# plot sea surface salnity
ax.pcolormesh(lon, lat, sss,
              cmap=cmocean.cm.haline, transform=ccrs.PlateCarree())

# plot a point at the flower garden banks.
x, y = projection.transform_point(-93.0-45.0/60.0, 27.0+55.0/60.0, 
                                  ccrs.PlateCarree())
ax.plot(x, y, 'ro', ms=30, alpha=0.3)

# set the extent
ax.set_extent([-98, -88, 27, 30.5], ccrs.PlateCarree())

# Note, gridlabels only work for mercator. See cartopy docs.
ax.gridlines()

plt.show()